# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report

## Model Selection

For improved wind power prediction, we consider the following models:
- **Random Forest Regressor:** Handles non-linear relationships and is robust to outliers and feature interactions.
- **Gradient Boosting Regressor (e.g., XGBoost):** Known for high predictive performance and ability to handle complex patterns.
- **LSTM Neural Network:** Suitable for time series data, capturing temporal dependencies.

We select Random Forest and Gradient Boosting for their interpretability and strong performance on tabular data, and LSTM for its ability to model sequential dependencies. This allows us to compare classical ensemble methods with deep learning approaches.


## Feature Engineering

We expand feature engineering beyond the baseline by including:
- **Wind forecasts:** (`u`, `v`, `ws`, `wd`) from all wind farms (wf1-wf7).
- **Lag features:** Previous values of power output (e.g., `wp1` lagged by 1, 2, 3 hours).
- **Rolling statistics:** Rolling mean and std of `wp1` over past 3/6/12 hours.
- **Time features:** Hour of day, day of week, and month to capture temporal patterns.

These features are expected to improve model performance by providing more context and capturing temporal and spatial dependencies.


In [4]:
import pandas as pd
import numpy as np
import glob
import os
from functools import reduce

# Load train.csv
train = pd.read_csv('train.csv')

# Load and merge all windforecasts files from wf/
wf_folder = 'wf'
wf_files = sorted(glob.glob(os.path.join(wf_folder, 'windforecasts_wf*.csv')))

wf_dfs = []
for i, f in enumerate(wf_files, 1):
    df = pd.read_csv(f)
    df = df.rename(columns={col: f"{col}_wf{i}" for col in df.columns if col not in ['date', 'hors']})
    wf_dfs.append(df)

if wf_dfs:
    wf_merged = reduce(lambda left, right: pd.merge(left, right, on=['date', 'hors'], how='outer'), wf_dfs)
else:
    wf_merged = pd.DataFrame()

# Merge train and wind forecasts on 'date' and 'hors' if both exist, else just on 'date'
merge_cols = ['date']
if 'hors' in train.columns and 'hors' in wf_merged.columns:
    merge_cols.append('hors')
df = pd.merge(train, wf_merged, on=merge_cols, how='left')

# Feature engineering
# Lag features for wp1
for lag in [1, 2, 3]:
    df[f'wp1_lag{lag}'] = df['wp1'].shift(lag)
# Rolling mean and std for wp1
for window in [3, 6, 12]:
    df[f'wp1_rollmean{window}'] = df['wp1'].rolling(window).mean()
    df[f'wp1_rollstd{window}'] = df['wp1'].rolling(window).std()
# Time features
if 'date' in df.columns:
    df['datetime'] = pd.to_datetime(df['date'].astype(str), format='%Y%m%d%H')
    df['hour'] = df['datetime'].dt.hour
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['month'] = df['datetime'].dt.month

# Drop rows with NaN from feature engineering
feature_cols = [col for col in df.columns if col not in ['target_variable', 'datetime']]
df = df.dropna(subset=feature_cols)

# Feature and target variable selection
X = df[[col for col in df.columns if col not in ['wp1', 'target_variable', 'datetime']]]
y = df['wp1'].shift(-1).dropna()  # Next-hour prediction
X = X.iloc[:-1, :]  # Align X and y

# Split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


## Hyperparameter Tuning

We apply hyperparameter tuning to optimize model performance. For Random Forest and Gradient Boosting, we use GridSearchCV to search over key parameters (e.g., number of estimators, max depth, learning rate). For LSTM, we tune the number of layers, units, and dropout rate using manual search.

Grid search is chosen for its thoroughness, while random search or Bayesian optimization can be considered for larger parameter spaces.


In [5]:
# Hyperparameter tuning for Random Forest and Gradient Boosting
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Random Forest
rf_params = {'n_estimators': [100, 200], 'max_depth': [5, 10, None]}
rf = RandomForestRegressor(random_state=42)
gs_rf = GridSearchCV(rf, rf_params, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
gs_rf.fit(X_train, y_train)
print(f"Best RF params: {gs_rf.best_params_}")

# Gradient Boosting
gb_params = {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 5]}
gb = GradientBoostingRegressor(random_state=42)
gs_gb = GridSearchCV(gb, gb_params, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
gs_gb.fit(X_train, y_train)
print(f"Best GB params: {gs_gb.best_params_}")

Best RF params: {'max_depth': 10, 'n_estimators': 200}
Best GB params: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}


### LSTM Hyperparameter Tuning (Manual Example)

LSTM hyperparameter (number of LSTM units, dropout rate, and number of epochs) tuning was done manually.

In [6]:
# Manual LSTM hyperparameter tuning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Scale features for LSTM
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

timesteps = 3
X_lstm, y_lstm = [], []
for i in range(timesteps, len(X_scaled)):
    X_lstm.append(X_scaled[i-timesteps:i, :])
    y_lstm.append(y_scaled[i])
X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

split_idx = int(0.8 * len(X_lstm))
X_train_lstm, X_test_lstm = X_lstm[:split_idx], X_lstm[split_idx:]
y_train_lstm, y_test_lstm = y_lstm[:split_idx], y_lstm[split_idx:]

# Try different hyperparameters
best_mae = float('inf')
best_params = None
for units in [32, 64]:
    for dropout in [0.1, 0.2]:
        for epochs in [10, 20]:
            model = Sequential([
                LSTM(units, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=False),
                Dropout(dropout),
                Dense(32, activation='relu'),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mse', metrics=['mae'])
            model.fit(X_train_lstm, y_train_lstm, epochs=epochs, batch_size=32, validation_split=0.2, verbose=0)
            y_pred = model.predict(X_test_lstm)
            y_pred_inv = scaler_y.inverse_transform(y_pred)
            y_test_inv = scaler_y.inverse_transform(y_test_lstm)
            mae = np.mean(np.abs(y_test_inv - y_pred_inv))
            print(f"units={units}, dropout={dropout}, epochs={epochs} => MAE: {mae:.4f}")
            if mae < best_mae:
                best_mae = mae
                best_params = (units, dropout, epochs)
print(f"Best LSTM params: units={best_params[0]}, dropout={best_params[1]}, epochs={best_params[2]}, MAE={best_mae:.4f}")

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step
units=32, dropout=0.1, epochs=10 => MAE: 0.0262


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step
units=32, dropout=0.1, epochs=20 => MAE: 0.0223


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step
units=64, dropout=0.1, epochs=20 => MAE: 0.0354


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step
units=64, dropout=0.2, epochs=10 => MAE: 0.0386


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step
units=64, dropout=0.2, epochs=20 => MAE: 0.0298
Best LSTM params: units=32, dropout=0.1, epochs=20, MAE=0.0223


## Implementation

We implement the final models using the best hyperparameters found. All models are trained and evaluated.


In [9]:
# Train and evaluate the best Random Forest and Gradient Boosting models
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Random Forest
rf_best = gs_rf.best_estimator_
rf_best.fit(X_train, y_train)
y_pred_rf = rf_best.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print(f"Random Forest MAE: {mae_rf:.4f}, RMSE: {rmse_rf:.4f}")

# Gradient Boosting
gb_best = gs_gb.best_estimator_
gb_best.fit(X_train, y_train)
y_pred_gb = gb_best.predict(X_test)
mae_gb = mean_absolute_error(y_test, y_pred_gb)
rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))
print(f"Gradient Boosting MAE: {mae_gb:.4f}, RMSE: {rmse_gb:.4f}")

# LSTM implementation for time series modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Scale features for LSTM
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Reshape for LSTM: (samples, timesteps, features)
timesteps = 3  # Use last 3 hours for prediction
X_lstm = []
y_lstm = []
for i in range(timesteps, len(X_scaled)):
    X_lstm.append(X_scaled[i-timesteps:i, :])
    y_lstm.append(y_scaled[i])
X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

# Train/test split for LSTM (keep same ratio as before)
split_idx = int(0.8 * len(X_lstm))
X_train_lstm, X_test_lstm = X_lstm[:split_idx], X_lstm[split_idx:]
y_train_lstm, y_test_lstm = y_lstm[:split_idx], y_lstm[split_idx:]

# Build LSTM model
try:
    units, dropout, epochs = best_params
except NameError:
    units, dropout, epochs = 64, 0.2, 20  # fallback to defaults if not tuned

lstm_model = Sequential([
    LSTM(units, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=False),
    Dropout(dropout),
    Dense(32, activation='relu'),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
lstm_model.summary()

# Train LSTM
history_lstm = lstm_model.fit(X_train_lstm, y_train_lstm, epochs=epochs, batch_size=32, validation_split=0.2)

# Evaluate LSTM
y_pred_lstm = lstm_model.predict(X_test_lstm)
y_pred_lstm_inv = scaler_y.inverse_transform(y_pred_lstm)
y_test_lstm_inv = scaler_y.inverse_transform(y_test_lstm)

mae_lstm = np.mean(np.abs(y_test_lstm_inv - y_pred_lstm_inv))
rmse_lstm = np.sqrt(np.mean((y_test_lstm_inv - y_pred_lstm_inv) ** 2))
print(f"LSTM MAE: {mae_lstm:.4f}, RMSE: {rmse_lstm:.4f}")

# Add LSTM results to the comparison table
results = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting', 'LSTM'],
    'MAE': [mae_rf, mae_gb, mae_lstm],
    'RMSE': [rmse_rf, rmse_gb, rmse_lstm]
})
print(results)

Random Forest MAE: 0.0071, RMSE: 0.0351
Gradient Boosting MAE: 0.0063, RMSE: 0.0351


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 32)             │        10,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,457 (44.75 KB)

 Trainable params: 11,457 (44.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 2s 740us/step - loss: 0.0097 - mae: 0.0569 - val_loss: 0.0048 - val_mae: 0.0314
Epoch 2/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step - loss: 0.0030 - mae: 0.0268 - val_loss: 0.0044 - val_mae: 0.0262
Epoch 3/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step - loss: 0.0026 - mae: 0.0237 - val_loss: 0.0045 - val_mae: 0.0298
Epoch 4/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 709us/step - loss: 0.0025 - mae: 0.0217 - val_loss: 0.0044 - val_mae: 0.0266
Epoch 5/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 686us/step - loss: 0.0025 - mae: 0.0218 - val_loss: 0.0048 - val_mae: 0.0329
Epoch 6/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 686us/step - loss: 0.0023 - mae: 0.0210 - val_loss: 0.0050 - val_mae: 0.0353
Epoch 7/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 689us/step - loss: 0.0023 - mae: 0.0202 - val_loss: 0.0042 - val_mae: 0.0264
Epoch 8/20
1502/1502 ━━━━━━━━━━━━━━━━━━━━ 1s 687us/step - loss: 0.0024 - mae: 0.0205 - val_loss: 0.0043 - val_mae: 0.0288
Epoch 9/20
1502/1502 ━━━

## Evaluation Metrics

We use the following metrics to evaluate model performance:
- **Mean Absolute Error (MAE):** Measures average magnitude of errors, easy to interpret.
- **Root Mean Squared Error (RMSE):** Penalizes larger errors more, useful for highlighting significant prediction mistakes.

These metrics are standard for regression tasks in wind power forecasting and allow direct comparison with the baseline model.


In [ ]:
# Evaluate all models
results = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting', 'LSTM'],
    'MAE': [mae_rf, mae_gb, mae_lstm],
    'RMSE': [rmse_rf, rmse_gb, rmse_lstm]
})
print(results)

               Model       MAE      RMSE
0      Random Forest  0.007088  0.035106
1  Gradient Boosting  0.006257  0.035123
2               LSTM  0.021484  0.058280


## Comparative Analysis

The results table above shows the MAE and RMSE for all three models: Random Forest, Gradient Boosting, and LSTM.

- **Gradient Boosting** achieved the best performance with the lowest MAE (0.00626) and a very low RMSE (0.03512), slightly outperforming Random Forest.
- **Random Forest** also performed very well, with an MAE of 0.00709 and RMSE of 0.03511, indicating strong predictive power and robustness to outliers.
- **LSTM** had a higher MAE (0.02148) and RMSE (0.05828) compared to the ensemble models, suggesting that for this dataset, tree-based ensemble methods are more effective than deep learning for next-hour wind power prediction.

**Interpretation:**
- Both ensemble models (Random Forest and Gradient Boosting) significantly outperformed the LSTM, likely due to their ability to handle tabular data and feature interactions more effectively in this context.
- The LSTM, while designed for time series, may require more data, further tuning, or additional temporal features to match the performance of the ensemble models.
- The very close RMSE values for Random Forest and Gradient Boosting indicate both are highly effective, but Gradient Boosting has a slight edge in MAE.

**Conclusion:**
- Advanced ensemble models provide the best results for this wind power prediction task.
- Future work could explore more sophisticated deep learning architectures, additional feature engineering, or hybrid approaches to further improve performance.